## Real time visualization

In [ ]:
include("src/mcmc.jl")
include("src/visualization.jl");

### Setup sliders to tune parameters

In [ ]:
# Gives us some sliders to manipulate parameters
temperature_slider = slider(0.:0.01:10., label="temperature", value=2.26)
temperature = observe(temperature_slider)
temperature_signal = Signal(2.26)
on(val->push!(temperature_signal,val), temperature)
display(temperature_slider)

h_slider = slider(-5.:0.1:+5., label="field")
h = observe(h_slider)
h_signal = Signal(0.)
on(val->push!(h_signal,val), h)
display(h_slider)

a_slider = slider(0.:0.1:2., label="simulation speed", value=0.)
a = observe(a_slider)
a_signal = Signal(0.)
on(val->push!(a_signal,val), a)
display(a_slider)

__Prepare the initial state__

*Make sure to call `reset_window()` before changing the system size!*

In [ ]:
reset_window()
L=128
config0 = frustratedConfiguration(L);
cluster = zeros(Bool, L,L);

__Prepare the window, signals and primitives. Start the renderloop.__

In [ ]:
if !isdefined(:window) || !isopen(window)
    window=glscreen(resolution=(800,800))
end

reset_window()

target_fps = 60
timesignal = fps(target_fps)

state_map = Reactive.map((_,T,h)->sweep!(config0,div(a[]*L^2,target_fps),1/T,h), timesignal,temperature_signal,h_signal);
color_signal=map(_->reshape(color_gen(config0,0.3),L^2,1)[:,1], timesignal)

#position = Point2f0[Point2f0(800/L*(xi+1/2),800/L*(yi+1/2)) for (xi,yi) in product(0:L-1,0:L-1)]
position = Point3f0[Point3f0(2*xi/L-1.,2*yi/L-1.,0) for (xi,yi) in product(0:L-1,0:L-1)]

#square = HyperRectangle(Vec2f0(0),Vec2f0(800/L));
square = HyperRectangle(Vec3f0(0.),Vec3f0(2/L,2/L,0));
circle = HyperSphere(Point3f0(0),2f0/L);

# [diffuse_color, specular_color, ambient_color, position]
light  = Vec3f0[Vec3f0(0.0,0.0,0.0), Vec3f0(0.1,0.1,0.1), Vec3f0(1.0,1.0,1.0), Vec3f0(0,0,100)];

light=Signal(light)

lattice = visualize((square,position),color=color_signal);
lattice.children[1].uniforms[:light] = light;

_view(lattice,window,camera=:perspective)

adjust_cam!(window,eyepos_vec=Vec3f0(0,0,-1))

@async renderloop(window)